# Pulmonary embolism
Pulmonary embolism is a blockage in one of the pulmonary arteries in your lungs. In most cases, pulmonary embolism is caused by blood clots that travel to the lungs from deep veins in the legs or, rarely, from veins in other parts of the body (deep vein thrombosis).

Because the clots block blood flow to the lungs, pulmonary embolism can be life-threatening. However, prompt treatment greatly reduces the risk of death. Taking measures to prevent blood clots in your legs will help protect you against pulmonary embolism.
![img](https://www.mayoclinic.org/-/media/kcms/gbs/patient-consumer/images/2013/11/15/17/38/ds00223_-ds00429_-ds01005_im01276_mcdc7_pulmonaryembolismthu_jpg.jpg)
# Symptoms
Pulmonary embolism symptoms can vary greatly, depending on how much of your lung is involved, the size of the clots, and whether you have underlying lung or heart disease.

Common signs and symptoms include:

* Shortness of breath. This symptom typically appears suddenly and always gets worse with exertion.
* Chest pain. You may feel like you're having a heart attack. The pain is often sharp and felt when you breathe in deeply, often stopping you from being able to take a deep breath. It can also be felt when you cough, bend or stoop.
* Cough. The cough may produce bloody or blood-streaked sputum.

Other signs and symptoms that can occur with pulmonary embolism include:

* Rapid or irregular heartbeat
* Lightheadedness or dizziness
* Excessive sweating
* Fever
* Leg pain or swelling, or both, usually in the calf caused by a deep vein thrombosis
* Clammy or discolored skin (cyanosis)

In [ ]:
!conda install -c conda-forge gdcm -y
!pip install pandas-profiling -y

In [ ]:
# Setup
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from matplotlib import animation, rc
import seaborn as sns

sns.set_style('darkgrid')
import pydicom as dcm
import scipy.ndimage
import gdcm
import glob
import imageio
from IPython import display

from skimage import measure 
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from skimage.morphology import disk, opening, closing
from tqdm import tqdm

from IPython.display import HTML
from PIL import Image

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

from os import listdir, mkdir

path = "../input/rsna-str-pulmonary-embolism-detection/"
files = glob.glob(path+'/train/*/*/*.dcm')


In [ ]:
# Reading Data
train = pd.read_csv(path + "train.csv")
test = pd.read_csv(path + "test.csv")
print("Train Data Shape:",train.shape)
print("Test Data Shape:",test.shape)

In [ ]:
train.head(5).T


In [ ]:
train.info()


In [ ]:
print('Missing values in train data:',train.isnull().sum().sum())
print('Missing values in test data:',test.isnull().sum().sum())

In [ ]:
from pandas_profiling import ProfileReport
profile = ProfileReport(train, title='Training Data Report')

# EDA

In [ ]:
profile.to_notebook_iframe()

In [ ]:
cols = train.copy()
cols.drop(['StudyInstanceUID','SeriesInstanceUID','SOPInstanceUID'],axis=1,inplace=True)
columns = cols.columns

corr = cols.corr()
mask = np.zeros_like(corr)
mask[np.triu_indices_from(mask)] = True
with sns.axes_style("white"):
    f, ax = plt.subplots(figsize=(12, 12))
    ax = sns.heatmap(corr,mask=mask,square=True,linewidths=.8,cmap="gnuplot",annot=True)

In [ ]:
fig, ax = plt.subplots(7,2,figsize=(16,28))
for i,col in enumerate(columns): 
    plt.subplot(7,2,i+1)
    sns.countplot(cols[col],palette="gnuplot")   

# what is dicom?
igital Imaging and Communications in Medicine (DICOM) - an international standard related to the exchange, storage and communication of digital medical images. Prior to this format, there was no standardized way to transfer medical scans. So loading up a single patient's study outside the hospital, in older formats took about 10-30 minutes for a single scan!

While DICOM 16-bit images (with values ranging from -32768..32767), other 8-bit greyscale images store values 0 - 255. These value ranges in DICOM are useful, as they correlate with the Hounsfield Scale. Each voxel can store a large amount of information.

Read more about it here: https://www.dicomstandard.org/

![img](https://media.giphy.com/media/2kWaWmhoFfFkI/giphy.gif)



In [ ]:
from random import randint
# This function reads dicom images from the given path
def load_dicom(path):
    files = listdir(path)
    f = [dcm.dcmread(path + "/" + str(file)) for file in files]
    return f

random_integer = randint(0,len(train))
example = path + "train/" + train.StudyInstanceUID.values[random_integer] +'/'+ train.SeriesInstanceUID.values[random_integer]
scans = load_dicom(example)
scans[randint(0,len(scans))]

In [ ]:
f, plots = plt.subplots(3, 3, sharex='col', sharey='row', figsize=(15, 15))
for i in range(9):
    plots[i // 3, i % 3].axis('off')
    plots[i // 3, i % 3].imshow(dcm.dcmread(np.random.choice(files[:3000])).pixel_array,cmap='gist_earth_r')

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
ax.imshow(dcm.dcmread(np.random.choice(files[:3000])).pixel_array, cmap="gist_earth_r")

In [ ]:
def load_slice(path):
    slices = [dcm.read_file(path + '/' + s) for s in listdir(path)]
    slices.sort(key = lambda x: float(x.ImagePositionPatient[2]))
    try:
        slice_thickness = np.abs(slices[0].ImagePositionPatient[2] - slices[1].ImagePositionPatient[2])
    except:
        slice_thickness = np.abs(slices[0].SliceLocation - slices[1].SliceLocation)
        
    for s in slices:
        s.SliceThickness = slice_thickness
        
    return slices

def transform_to_hu(slices):
    images = np.stack([file.pixel_array for file in slices])
    images = images.astype(np.int16)

    # convert ouside pixel-values to air:
    # I'm using <= -1000 to be sure that other defaults are captured as well
    images[images <= -1000] = 0
    
    # convert to HU
    for n in range(len(slices)):
        
        intercept = slices[n].RescaleIntercept
        slope = slices[n].RescaleSlope
        
        if slope != 1:
            images[n] = slope * images[n].astype(np.float64)
            images[n] = images[n].astype(np.int16)
            
        images[n] += np.int16(intercept)
    
    return np.array(images, dtype=np.int16)
first_patient = load_slice(path+'/train/0003b3d648eb/d2b2960c2bbf')
first_patient_pixels = transform_to_hu(first_patient)

fig, plots = plt.subplots(8, 10, sharex='col', sharey='row', figsize=(20, 16))
for i in range(80):
    plots[i // 10, i % 10].axis('off')
    plots[i // 10, i % 10].imshow(first_patient_pixels[i], cmap="gray") 

In [ ]:

imageio.mimsave("/tmp/gif.gif", first_patient_pixels, duration=0.08)
display.Image(filename="/tmp/gif.gif", format='png')

# Prevention
Preventing clots in the deep veins in your legs (deep vein thrombosis) will help prevent pulmonary embolism. For this reason, most hospitals are aggressive about taking measures to prevent blood clots, including:

* Blood thinners (anticoagulants). These medications are often given to people at risk of clots before and after an operation — as well as to people admitted to the hospital with medical conditions, such as heart attack, stroke or complications of cancer.
* Compression stockings. Compression stockings steadily squeeze your legs, helping your veins and leg muscles move blood more efficiently. They offer a safe, simple and inexpensive way to keep blood from stagnating during and after general surgery.

* Leg elevation. Elevating your legs when possible and during the night also can be very effective. Raise the bottom of your bed 4 to 6 inches (10 to 15 cm) with blocks or books.

* Physical activity. Moving as soon as possible after surgery can help prevent pulmonary embolism and hasten recovery overall. This is one of the main reasons your nurse may push you to get up, even on your day of surgery, and walk despite pain at the site of your surgical incision.

* Pneumatic compression. This treatment uses thigh-high or calf-high cuffs that automatically inflate with air and deflate every few minutes to massage and squeeze the veins in your legs and improve blood flow.